In [ ]:
! pip install -r requirements.txt

: 

In [ ]:
from paddleocr import PaddleOCR
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np

: 

In [37]:
ocr = PaddleOCR(
    lang="korean",
    det_db_thresh=0.2,
    det_db_box_thresh=0.4,
    use_angle_cls=True
)

[2025/11/12 17:34:57] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\lavalab/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.2, det_db_box_thresh=0.4, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\lavalab/.paddleocr/whl\\rec\\korean\\korean_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48

In [35]:
img_path = './Image/handwriting.png'

In [38]:
# --- 2. OCR 실행 ---
print("OCR을 실행합니다...")
result = ocr.ocr(img_path, cls=True)
print("OCR 실행 완료.")

# --- 3. 결과 확인 (가장 중요!) ---
# result[0]이 None인지 (글자를 못 찾았는지) 확인합니다.
if result is None or result[0] is None:
    print(f"이미지에서 글자를 찾지 못했습니다: {img_path}")
else:
    # --- 4. 데이터 추출 (새로운 형식에 맞게 수정) ---
    # data는 [ [box, (text, score)], [box, (text, score)], ... ] 형태입니다.
    data = result[0]
    
    boxes = [line[0] for line in data]
    texts = [line[1][0] for line in data]
    scores = [line[1][1] for line in data]

    print("======================")
    for i, (text, score) in enumerate(zip(texts, scores)):
        print(f"{i+1}. {text} (confidence: {score:.2f})")
    print("======================")

    # --- 5. 이미지에 결과 그리기 ---
    image = Image.open(img_path).convert('RGB')
    draw = ImageDraw.Draw(image)

    # 폰트 경로 수정 (Windows '맑은 고딕' 기준)
    font_path_windows = "C:/Windows/Fonts/malgun.ttf"
    try:
        if os.path.exists(font_path_windows):
            font = ImageFont.truetype(font_path_windows, 25)
        else:
            # 맑은 고딕이 없으면 기본 폰트
            font = ImageFont.load_default()
    except Exception as e:
        print(f"폰트 로딩 실패 (기본 폰트 사용): {e}")
        font = ImageFont.load_default()


    for text, score, box in zip(texts, scores, boxes):
        # box는 이미 [ [x,y], [x,y], [x,y], [x,y] ] 형태입니다.
        box_points = [(int(point[0]), int(point[1])) for point in box]
        draw.polygon(box_points, outline='green', width=2)
        
        # 텍스트 위치 (첫 번째 꼭짓점 위로 25픽셀)
        text_pos = (box_points[0][0], box_points[0][1] - 25)
        
        # 텍스트 그리기
        draw.text(text_pos, 
                  f'{text} ({score:.2f})', 
                  fill='red', font=font)

    # --- 6. 결과 저장 ---
    output_path = './test_result_improved1.jpg'
    image.save(output_path)
    print(f"'done'. 결과가 {output_path} 에 저장되었습니다.")

OCR을 실행합니다...
[2025/11/12 17:35:01] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.04709577560424805
[2025/11/12 17:35:01] ppocr DEBUG: cls num  : 1, elapsed : 0.030330896377563477
[2025/11/12 17:35:01] ppocr DEBUG: rec_res num  : 1, elapsed : 0.18255853652954102
OCR 실행 완료.
이미지에서 글자를 찾지 못했습니다: ./Image/handwriting.png
